In [1]:
import os
import time
from logger import setup_logger
import requests
import numpy as np

# ロガー設定
logging = setup_logger("dummyClient")

def log_meesage(message):
    print(message)
    logging.info(message)

def log_error(message):
    print("error: {}".format(message))
    logging.error(message)


# HOST = "192.168.0.3" #津の設定
HOST = "10.178.64.66" #debug
PORT = 5000

PROXY = ""


# Flask設定
flask_root = "http://" + HOST + ":" + str(PORT) + "/"

log_meesage("flask root:" + flask_root)

def log_meesage(message):
    print(message)
    logging.info(message)

def log_error(message):
    print("error: {}".format(message))
    logging.error(message)

def request_post(_act="", _data="", _value=None):
    act = _act
    try:
        if(_value is not None):
            prompt = {_data:str(_value)}
        else:
            prompt = {"":None,}
        # print(act, prompt)
        res = requests.post(url=flask_root+act, data=prompt, proxies={"http":PROXY})
        # log_meesage("Action Request: {} & {}, Response: {}".format(act, prompt, res))
        return res
    
    except Exception as e: #接続エラー時。（サーバー側との接続が出来ないときなど）
        log_error("Error in request_post: {}".format(e))

def request_posts(_act="", _data="", _value=None):
    act = _act
    try:
        if(_value is not None):
            prompt = {}
            for d, v in zip(_data, _value):
                prompt[d] = str(v)
            # prompt = {_data:str(_value)}
        else:
            prompt = {"":None,}
        # print(act, prompt)
        res = requests.post(url=flask_root+act, data=prompt, proxies={"http":PROXY})
        # log_meesage("Action Request: {} & {}, Response: {}".format(act, prompt, res))
        return res
    
    except Exception as e: #接続エラー時。（サーバー側との接続が出来ないときなど）
        log_error("Error in request_post: {}".format(e))

def check_res(res):
    if(res.status_code==200): # サーバーとの通信はできてる
        # log_meesage("{},  {}".format(res.status_code, res.text))
        if(res.text == "False"): # ロボットに与えた指示が失敗
            return False
        else:
            return True
    else:
        log_error("{},  {}".format(res.status_code, res.text))
        return False
    
def loadSAMModel(target=None):
    try:
        image_path = "rot_shift"
        model_path = os.path.join("rot_shift", target)
        sam_type = "vit_b"
        log_meesage("sam image_save_path: {}".format(image_path))
        res = request_posts(_act="loadSAMModel", _data=["image_save_path", "model_path", "sam_type"], _value=[image_path,model_path,sam_type])
        if(check_res(res)):
            log_meesage("SAM Model Loaded")
        else:
            log_error("SAM Model Loading Failed")

    except Exception as e:
        log_error("Error in load SAM Model: {}".format(e))

flask root:http://10.178.64.66:5000/


In [2]:
def request(image_dir, file_name, repeat):
    s_time = time.time()
    res = request_posts(_act="image_rot_shift", _data=["image_dir", "file_name", "repeat"], _value=[image_dir, file_name, repeat])
    if(check_res(res)):
        output = eval(res.text)
        x = output[0]
        y = output[1]
        rot_angle = output[2]
        # log_meesage(f"[time:{time.time()-s_time}] Image shit x:{x} y:{y}, rot_angle:{rot_angle}")
        return x,y,rot_angle
    else:
        print("ERROR")
        return False, False, False

In [3]:
import cv2
import os
import matplotlib.pyplot as plt
import csv

image_dir = "test_data/20250717/003_high_angle"

file_name = "20250717_130444314_003_000.png"

repeat = 3


In [4]:
from tqdm import tqdm
def execute(image_dir, repeat=2, failed_list = None):
    tmp = ""
    if failed_list is not None:
        tmp = "analysis"
        image_list = failed_list
    else:
        image_list = os.listdir(image_dir)
    
    failed_list = []

    for test_image_path in tqdm(image_list):
        if tmp == "analysis":
            file_name = test_image_path[1]
        else: 
            file_name = test_image_path
        
        # print(f"image_path: {image_dir}, file_name: {file_name}")
        image = cv2.imread(os.path.join(image_dir, file_name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # plt.figure()
        # plt.axis("off")
        # plt.imshow(image)
        # plt.show()
        fig = plt.figure(figsize=(5, 1))
        plt.subplot(1,4,1)
        plt.axis("off")
        plt.title("original", fontsize=8)
        plt.imshow(image)
     
        x,y,rot_angle = request(image_dir, file_name, repeat)

        num = 0
        for _x, _y, _rot_angle in zip(x,y,rot_angle):
            if _x==0 and _y ==0:
                # print("next")
                break    
            # print("RESULT",(_x, _y), _rot_angle)
            rot_mat = cv2.getRotationMatrix2D((_x, _y), _rot_angle, 1)
            height, width = image.shape[:2]

            affine_image = cv2.warpAffine(image, rot_mat, (width, height))

            mv_mat = np.float32([[1, 0, -_x+200],[0, 1, -_y+200]])
            affine_image = cv2.warpAffine(affine_image, mv_mat, (width, height))

            # plt.figure()
            # plt.axis("off")
            # plt.imshow(affine_image)
            # plt.show()
            plt.subplot(1,4,num+2)
            plt.axis("off")
            plt.imshow(affine_image)
            plt.title(f"result_{num+1}", fontsize=8)
            save_path = os.path.join(*["result",tmp,image_dir])
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            plt.savefig(os.path.join(save_path, file_name))
            num += 1

        failed_flag = False
        if(num==0):
            save_path = os.path.join(*["result",tmp,image_dir])
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            plt.savefig(os.path.join(save_path, file_name))
            failed_flag = True
        elif(num==1 and repeat==2):
            failed_flag = True
        elif(num==2 and repeat==3):
            failed_flag = True
        plt.close()
        if failed_flag:
            failed_list.append([num, file_name])
    return failed_list

In [5]:
image_dir = "test_data/2025_07_17_triple_black/solution002_solution/003_high_angle"
loadSAMModel(target="advance_triple_black")
failed_list = execute(image_dir, repeat=3)
print(failed_list)

with open('test_data/2025_07_17_triple_black/failed_list.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(failed_list)



sam image_save_path: rot_shift
SAM Model Loaded


100%|██████████| 164/164 [12:15<00:00,  4.49s/it]

[[2, '20250717_112805967_003_000.png'], [0, '20250717_121615254_003_000.png'], [2, '20250717_112250786_003_000.png'], [0, '20250717_112849748_003_000.png'], [2, '20250717_121537155_003_000.png'], [0, '20250717_113941031_003_000.png'], [2, '20250717_113923692_003_000.png'], [2, '20250717_111637515_003_000.png'], [2, '20250717_114510306_003_000.png'], [2, '20250717_121551053_003_000.png'], [2, '20250717_111659285_003_000.png'], [2, '20250717_111631823_003_000.png'], [0, '20250717_112310226_003_000.png'], [0, '20250717_113710582_003_000.png'], [0, '20250717_113644820_003_000.png'], [2, '20250717_112816494_003_000.png'], [2, '20250717_113901075_003_000.png'], [0, '20250717_114549240_003_300.png'], [0, '20250717_121630282_003_000.png'], [2, '20250717_112800834_003_000.png'], [2, '20250717_113920914_003_000.png'], [0, '20250717_113658112_003_000.png'], [2, '20250717_113605642_003_000.png'], [0, '20250717_111720143_003_000.png'], [0, '20250717_111801457_003_000.png'], [0, '20250717_114528514_

In [ ]:
image_dir = "test_data/2025_07_17_double_white/solution002_solution/003_high_angle"
loadSAMModel(target="advance_double_white")
failed_list = execute(image_dir, repeat=2)
print(len(failed_list))
print(failed_list)
with open('test_data/2025_07_17_double_white/failed_list.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(failed_list)

sam image_save_path: rot_shift
SAM Model Loaded


100%|██████████| 144/144 [12:20<00:00,  5.15s/it]

0
[]


: 

In [5]:
image_dir = "test_data/003_0717_high_angle_triple_black_left"
loadSAMModel(target="advance_triple_black")
failed_list = execute(image_dir, repeat=3)
print(len(failed_list))
print(failed_list)
# with open('test_data/test_data/003_0717_high_angle_triple_black_left_failed_list.csv', 'w') as f:
#     writer = csv.writer(f)
#     writer.writerows(failed_list)

sam image_save_path: rot_shift
SAM Model Loaded


100%|██████████| 108/108 [14:26<00:00,  8.02s/it]

0
[]


In [5]:
image_dir = "test_data/003_0717_high_angle_triple_black_right"
loadSAMModel(target="advance_triple_black")
failed_list = execute(image_dir, repeat=3)
print(len(failed_list))
print(failed_list)
# with open('test_data/test_data/003_0717_high_angle_triple_black_right_failed_list.csv', 'w') as f:
#     writer = csv.writer(f)
#     writer.writerows(failed_list)


sam image_save_path: rot_shift
SAM Model Loaded


100%|██████████| 108/108 [13:31<00:00,  7.51s/it]

0
[]


In [ ]:
image_dir = "test_data/2025_07_17_double_white/solution002_solution/003_high_angle"
loadSAMModel(target="advance_double_white")
csv_dir = 'test_data/2025_07_17_double_white/failed_list.csv'
failed_list = []
with open(csv_dir) as f:
  for row in csv.reader(f):
    failed_list.append(row)
# print(len(failed_list))
# execute(image_dir, repeat=2, failed_list=failed_list)


sam image_save_path: rot_shift
SAM Model Loaded
10


100%|██████████| 10/10 [00:43<00:00,  4.39s/it]


[]